In [ ]:
import sys
sys.path.append('.')

import import_ipynb
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_community.chains import 
from DocumentLoader import DocumentLoader
from DocumentTextSplitterManager import DocumentTextSplitterManager
from EmbeddingManager import EmbeddingManager

In [2]:
documentloader = DocumentLoader()
loaded_documents = documentloader.load_directory(directory_path="data", glob_pattern="**/*.pdf")
print(loaded_documents)


Loading documents....


INFO:DocumentLoader:Total documents loaded 8


loaded documents
[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Everstorm  Outfitters    RETURN  &  EXCHANGE  POLICY    Document  ROX-2025-05   Easy-Fit  Promise    If  your  gear  doesn’t  fit  or  just  isn’t  your  vibe,  send  it  back  within  **30  days**  of  delivery  for  a  refund  or  free  size  exchange.   Eligibility  Checklist    ●  Unworn,  unwashed,  no  odors,  tags  attached    ●  Original  shoe  box  (footwear)  placed  inside  outer  carton    ●  Electronics  (power-banks,  headlamps)  unopened  unless  faulty   How  to  Start    ●  Visit  everstorm.example/returns  →  enter  order  #  and  email.    ●  Select  “Refund”  or  “Exchange.”    ●  Print  prepaid  label;  pack  securely.  Multiple  items  can  share  one  box.   Instant  Exchange  Hol

INFO:DocumentLoader:Total documents loaded 8


[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Everstorm  Outfitters    RETURN  &  EXCHANGE  POLICY    Document  ROX-2025-05   Easy-Fit  Promise    If  your  gear  doesn’t  fit  or  just  isn’t  your  vibe,  send  it  back  within  **30  days**  of  delivery  for  a  refund  or  free  size  exchange.   Eligibility  Checklist    ●  Unworn,  unwashed,  no  odors,  tags  attached    ●  Original  shoe  box  (footwear)  placed  inside  outer  carton    ●  Electronics  (power-banks,  headlamps)  unopened  unless  faulty   How  to  Start    ●  Visit  everstorm.example/returns  →  enter  order  #  and  email.    ●  Select  “Refund”  or  “Exchange.”    ●  Print  prepaid  label;  pack  securely.  Multiple  items  can  share  one  box.   Instant  Exchange  Hold    We  place  a

In [3]:
textSplitterManager = DocumentTextSplitterManager()
splitter = textSplitterManager.get_recursive_splitter()
chunks = splitter.split_documents(loaded_documents)
print(chunks)

INFO:DocumentTextSplitterManager:Created recursive character spliersize:100, overlap: 20


[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Everstorm  Outfitters    RETURN  &  EXCHANGE  POLICY    Document  ROX-2025-05   Easy-Fit  Promise'), Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Promise    If  your  gear  doesn’t  fit  or  just  isn’t  your  vibe'), Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Return_and_exchange_policy', 'source': 'data/Everstorm_Return_and_exchange_policy.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content=',  send  it  bac

In [4]:
embeddingManager = EmbeddingManager()
embeddings = embeddingManager.embeddings


<string>:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-small
INFO:EmbeddingManager:Initialized sentenceTransformer embeddings


In [5]:
vectorDb = FAISS.from_documents(chunks, embeddings)
query ="what Footwear Conversion do you make use of"

vectorDb.similarity_search(query,k=8)

retriever = vectorDb.as_retriever(search_kwargs= {"k": 8})





INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


In [8]:
llm = Ollama(model="gemma3:1b", temperature="0.5")
text = "Hi my name is ibrahim?"
response = llm.invoke(text)
print(response)

Hi Ibrahim! It's nice to meet you. 😊 

Is there anything I can help you with today?


In [ ]:
SYSTEM_TEMPLATE = """
You are a **Customer Support Chatbot**. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with “I'm not sure from the docs.”

Rules:
1) Use ONLY the provided <context> to answer.
2) If the answer is not in the context, say: "I don't know based on the retrieved documents."
3) Be concise and accurate. Prefer quoting key phrases from the context.
4) When possible, cite sources as [source: source] using the metadata.

CONTEXT:
{context}

USER:
{question}
"""

In [ ]:
prompt = PromptTemplate(input_variables=["context","question"],template=SYSTEM_TEMPLATE)

chain = ConversationChain.llm